# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült a   
https://github.com/bradleypallen/keras-movielens-cf oldalon található forrás alapján.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot MIT licensz védi.
</PRE>

## Importok

In [1]:
%matplotlib inline
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from CFModel import CFModel # letölthető innen: https://github.com/bradleypallen/keras-movielens-cf/blob/master/CFModel.py

Using TensorFlow backend.


## Konstansok


In [2]:
RATINGS_CSV_FILE = 'ml1m_ratings.csv'
MODEL_WEIGHTS_FILE = 'ml1m_weights.h5'
K_FACTORS = 120
RNG_SEED = 1446557

## MovieLens 1M adatok betöltése

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
print (len(ratings), 'értékelés betöltve.')

1000209 értékelés betöltve.


## Tanítóhalmaz előállítása

Felhasználó + film bemeneti, értékelés kimeneti adathármasokra van szükségünk.


In [4]:
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)
Users = shuffled_ratings['user_emb_id'].values
print ('Felhasználók:', Users, ', shape =', Users.shape)
Movies = shuffled_ratings['movie_emb_id'].values
print ('Filmek:', Movies, ', shape =', Movies.shape)
Ratings = shuffled_ratings['rating'].values
print ('Értékelések:', Ratings, ', shape =', Ratings.shape)

Felhasználók: [4403 1819 2571 ...,  354 3510  831] , shape = (1000209,)
Filmek: [2717 3692  149 ..., 2700 2920 3711] , shape = (1000209,)
Értékelések: [5 4 4 ..., 2 4 4] , shape = (1000209,)


## A modell

A modell forrása külön py fájlban található, ezt importáltuk is már.

In [5]:
model = CFModel(max_userid, max_movieid, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

/home/deep/Letöltések/vitmav45-master/2016/11/CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1)) # skaláris szorzat, maga az értékelés


## Tanítás

A tanítás maga hosszadalmas!

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = model.fit([Users, Movies], Ratings, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

Train on 900188 samples, validate on 100021 samples
Epoch 1/30
 - 522s - loss: 6.3302 - val_loss: 1.7864
Epoch 2/30
 - 507s - loss: 1.2853 - val_loss: 1.0468
Epoch 3/30
 - 511s - loss: 0.9504 - val_loss: 0.9090
Epoch 4/30
 - 520s - loss: 0.8615 - val_loss: 0.8545
Epoch 5/30
 - 507s - loss: 0.8162 - val_loss: 0.8245
Epoch 6/30
 - 508s - loss: 0.7847 - val_loss: 0.8050
Epoch 7/30
 - 506s - loss: 0.7584 - val_loss: 0.7883
Epoch 8/30
 - 515s - loss: 0.7337 - val_loss: 0.7780
Epoch 9/30
 - 521s - loss: 0.7095 - val_loss: 0.7680
Epoch 10/30
 - 527s - loss: 0.6858 - val_loss: 0.7583
Epoch 11/30
 - 491s - loss: 0.6618 - val_loss: 0.7533
Epoch 12/30
 - 503s - loss: 0.6377 - val_loss: 0.7486
Epoch 13/30
 - 508s - loss: 0.6133 - val_loss: 0.7462
Epoch 14/30


## RMSE görbék

In [7]:
loss = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                     'training': [ math.sqrt(loss) for loss in history.history['loss'] ],
                     'validation': [ math.sqrt(loss) for loss in history.history['val_loss'] ]})
ax = loss.ix[:,:].plot(x='epoch', figsize={7,10}, grid=True)
ax.set_ylabel("root mean squared error")
ax.set_ylim([0.0,3.0]);

NameError: name 'history' is not defined

## Print best validation RMSE

In [8]:
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

Minimum RMSE at epoch 15 = 0.8623
